In [1]:
import codecs
import os
import re

import pandas as pd
from pickle import dump ,load
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
os.chdir('./trainData')

In [3]:
data=pd.DataFrame(columns=['Text']) 
for folders in os.listdir(os.getcwd()):
    cnt=0
    for file in os.listdir(folders):
        data=data.append({'Text':codecs.open('./'+folders+'/'+file,encoding='utf8',\
                    errors='replace').read()},ignore_index=True)
        cnt+=1
    print(folders,cnt)

entertainment 384
sport 508
politics 414
tech 398
business 508


In [4]:
data.head()

,Text
0,Musicians to tackle US red tape\n\nMusicians' ...
1,Sideways dominates Spirit awards\n\nThe comedy...
2,Bennett play takes theatre prizes\n\nThe Histo...
3,Rock group Korn's guitarist quits\n\nThe guita...
4,Rocker Doherty in on-stage fight\n\nRock singe...


In [5]:
data=data.drop(data.index[0])

In [6]:
#data['Text']=data['Text'].apply(lambda x : re.sub("[(),!?@\'\`\"\_:.#%]",'',x))

In [7]:
data['Text'][:5]

1    Sideways dominates Spirit awards\n\nThe comedy...
2    Bennett play takes theatre prizes\n\nThe Histo...
3    Rock group Korn's guitarist quits\n\nThe guita...
4    Rocker Doherty in on-stage fight\n\nRock singe...
5    Manics in charge of BBC 6 Music\n\nThe Manic S...
Name: Text, dtype: object

In [8]:
data['head_lines']=data['Text'].apply(lambda x: 'startseq '+x.splitlines()[0]+' endseq')
data['Text']=data['Text'].apply(lambda x: 'startseq '+' '.join(x.splitlines()[1:])+' endseq')

In [10]:
data['Text'][1]

"startseq  The comedy Sideways has dominated this year's Independent Spirit Awards, winning all six of the awards for which it was nominated.  It was named best film while Alexander Payne won best director and best screenplay, along with writing partner Jim Taylor. It also won acting awards for stars Paul Giamatti, Thomas Haden Church and Virginia Madsen. Sideways is tipped to do well at Sunday's Oscars, with five nominations.  The awards, now in their 20th year, are given to films made outside the traditional studio system, and are traditionally held the day before the Oscars. Other winners included Catalina Sandino Moreno, who took best actress for her role as a drug smuggler in the Colombian drama Maria Full of Grace. Moreno is also nominated for best actress at the Oscars. The best first screenplay award went to Joshua Marston for Maria Full of Grace. Scrubs star Zach Braff won the award for best first feature for Garden State, which he wrote, directed and starred in. Oscar-nominat

In [11]:
data=data.sample(frac=1).reset_index(drop=True)

In [12]:
data.head()

,Text,head_lines
0,startseq One in 10 adult Americans - equivale...,startseq Millions buy MP3 players in US endseq
1,startseq India has signed a $40bn (£21bn) dea...,startseq India and Iran in gas export deal endseq
2,startseq If there is one thing certain to sti...,startseq Labour MPs' fears over squabbling endseq
3,startseq Peace protestors have lost a landmar...,startseq Peace demo appeal rejected endseq
4,"startseq Albania, Bulgaria and Macedonia has ...",startseq Go-ahead for Balkan oil pipeline endseq


In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
tokenizer=Tokenizer(filters='!"#$%&()*+,\'-./:;<=>?@[\\]^_`{|}~\t\n')

In [15]:
tokenizer.fit_on_texts(data['head_lines'].tolist()+data['Text'].tolist())

In [16]:
word_to_index=tokenizer.word_index
index_to_word={ index:word for word,index in word_to_index.items()}
vocab_size=len(word_to_index)
print(vocab_size)

29664


In [17]:
dump(word_to_index,open('word_to_index.pkl','wb'))

In [18]:
train_sequence_text=tokenizer.texts_to_sequences(data["Text"].tolist())
train_sequence_head=tokenizer.texts_to_sequences(data["head_lines"].tolist())

In [19]:
maxlen_x=max(len(i) for i in train_sequence_text)
maxlen_y=max(len(i) for i in train_sequence_head)
print(maxlen_x,maxlen_y)

4489 11


In [20]:
def data_for_model(sequences_head,sequences_text):
    x1=[]
    x2=[]
    y=[]
    for sequence_idx,sequence in enumerate(sequences_head):
        for idx in range(1,len(sequence)):
            in_seq,out_seq=sequence[:idx],sequence[idx]
            in_seq=pad_sequences([in_seq],maxlen=maxlen_y)[0]
            out_seq=to_categorical([out_seq],num_classes=vocab_size+1)[0]
            x1.append(pad_sequences([sequences_text[sequence_idx]],maxlen=maxlen_x)[0])
            x2.append(in_seq)
            y.append(out_seq)
    return np.array(x1),np.array(x2),np.array(y)
        


In [21]:
x1_train,x2_train,y_train=data_for_model(train_sequence_head,train_sequence_text)

In [46]:
dump(x1_train, open('x1_train.pkl', 'wb'))
dump(x2_train, open('x2_train.pkl', 'wb'))
dump(y_train, open('y_train.pkl', 'wb'))

In [29]:
os.getcwd()

'/home/aninda/protoTypeProject/nlp/Auto Headline Generator Using LSTM'

In [27]:
os.chdir('..')

In [31]:
embedding_dim=200
embedding_weights = np.empty((400000,200))

In [32]:
i=0
glove_index_dict = {}
with open('glove.6B.200d.txt','r') as file:
    for line in file:
        token=line.strip().split()
        glove_index_dict[token[0]] = i
        embedding_weights[i,:]=list(map(float,token[1:]))
        #print(i)
        i+=1

file.close()

In [35]:
np.random.seed(5)
embedding_weights*=.1

In [36]:
shape=(vocab_size+1,200)
scale = embedding_weights.std()*np.sqrt(12)/2
print(scale)
embedding = np.random.uniform(low=-scale, high=scale, size=shape)

0.0661404484678092


In [38]:
cnt=0
for word,idx in word_to_index.items():
    if word in glove_index_dict:
        embedding[idx,:] = embedding_weights[glove_index_dict[word],:]
        cnt+=1

In [39]:
activation_rnn_size = 40 
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False
seed=42
p_W, p_U, p_dense, weight_decay = 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [43]:
from tensorflow.keras.models import Sequential,Model,save_model
from tensorflow.keras.layers import Dense, Activation, Dropout, RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import add

In [44]:
regularizer = l2(0)

In [46]:

input1 = Input(shape=(maxlen_x,),name='input1')
text_em = Embedding(vocab_size+1, 200,weights=[embedding],embeddings_regularizer=regularizer,\
                    mask_zero=True,name='text_em')(input1)
text_drop=Dropout(.5,name='text_drop')(text_em)
#text_lstm1= LSTM(600,activation='relu',return_sequences=True,dropout=.5,kernel_regularizer=regularizer ,name='text_lstm')(text_drop)
#text_lstm2= LSTM(30,activation='relu',return_sequences=True,dropout=.5,kernel_regularizer=regularizer ,name='text_lstm1')(text_drop)
text_lstm3= LSTM(12,activation='relu',dropout=.5,kernel_regularizer=regularizer ,name='text_lstm2')(text_drop)


#encoder13 = RepeatVector(sum_txt_length)(encoder2)

input2 = Input(shape=(maxlen_y,),name='input2')
head_em = Embedding(vocab_size+1, 200,weights=[embedding],embeddings_regularizer=regularizer,\
                    mask_zero=True,name='head_em')(input2)
head_drop=Dropout(.5,name='head_drop')(head_em)
head_lstm1 = LSTM(12,dropout=.5,activation='relu',kernel_regularizer=regularizer ,name='head_lstm1')(head_drop)
#head_lstm2= LSTM(30,activation='relu',dropout=.5,kernel_regularizer=regularizer ,name='head_lstm2')(head_lstm1)
#encoder23 = RepeatVector(sum_txt_length)(encoder2)

decoder_add = add([text_lstm3,head_lstm1],name='decoder_add')
decoder_lstm= Dense(256,activation='relu',name='decoder_dence1')(decoder_add)
output = Dense(vocab_size+1, activation='softmax',name='decoder_dence2')(decoder_lstm)
# tie it together
model = Model(inputs=[input1,input2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

W0806 23:36:45.065155 140011615065920 deprecation.py:323] From /home/aninda/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:3868: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 4489)]       0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 11)]         0                                            
__________________________________________________________________________________________________
text_em (Embedding)             (None, 4489, 200)    5933000     input1[0][0]                     
__________________________________________________________________________________________________
head_em (Embedding)             (None, 11, 200)      5933000     input2[0][0]                     
______________________________________________________________________________________________

In [ ]:
model.fit([x1_train,x2_train],y_train,verbose=2,batch_size=20,epochs=5)

In [38]:
from keras.models import Model,load_model

In [35]:
os.chdir('..')

In [66]:
model1 = load_model('model.h5')

In [45]:
os.getcwd()

'/home/aninda/protoTypeProject/nlp/Auto Headline Generator Using LSTM'

In [47]:
test=pd.DataFrame(columns=['Text']) 
cnt=0
for file in os.listdir('./testData'):
    test=test.append({'Text':codecs.open('./testData/'+file,encoding='utf8',errors='replace').read()},ignore_index=True)
    cnt+=1
    print(file)

060.txt
026.txt
043.txt
207.txt
323.txt
063.txt
183.txt
057.txt
051.txt
111.txt
037.txt
119.txt
028.txt
001.txt
188.txt


In [48]:
test['head_lines']=test['Text'].apply(lambda x: 'startseq '+x.splitlines()[0]+' endseq')
test['Text']=test['Text'].apply(lambda x: 'startseq '+' '.join(list(filter(bool,x.splitlines()))[1:])+' endseq')

In [60]:
test_sequence_text=tokenizer.texts_to_sequences(test["Text"].tolist())
test1=pad_sequences([test_sequence_text[1]],maxlen=1247)

In [51]:
test1

array([[  0,   0,   0, ...,   6, 220,  25]], dtype=int32)

In [106]:
test1=pad_sequences([train_sequence_text[45]],maxlen=maxlen_x)
test1

array([[  0,   0,   0, ...,   1, 152,  19]], dtype=int32)

In [61]:
len(test1[0])

1247

In [64]:
def pred_description(model,tokenizer,doc,mx_len=9,):
    in_text='startseq'
    #print(type(tokenizer))
    for i in range(mx_len):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=mx_len)
        pred = model.predict([doc,sequence], verbose=0)
        pred=np.argmax(pred)
        print(pred)
        pred_word=index_to_word[pred]
        in_text+=' '+pred_word
        if pred_word=='endseq' or pred_word is None:
            break
    in_text=' '.join(in_text.split()[1:-1])
    return in_text

In [67]:
a=pred_description(model1,tokenizer,test1)

InvalidArgumentError: indices[0,958] = 13310 is not in [0, 6222)
	 [[Node: text_em_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](text_em_1/embeddings/read, text_em_1/Cast)]]

Caused by op 'text_em_1/Gather', defined at:
  File "/home/aninda/anaconda3/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/aninda/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/aninda/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aninda/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aninda/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/aninda/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloo

'endseq'

In [50]:

model = Sequential()
model.add(Embedding(vocab_size+1, 100,
                    input_length=59,
                    embeddings_regularizer=regularizer,  mask_zero=True,
                    name='embedding_1'))

for i in range(rnn_layers):
    lstm = LSTM(512, return_sequences=True, # batch_norm=batch_norm,
                kernel_regularizer=regularizer, recurrent_regularizer=regularizer,
                bias_regularizer=regularizer, dropout=p_W, recurrent_dropout=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))



In [51]:
def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlen_x, maxlenh=maxlen_y):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    #activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))

In [52]:
if activation_rnn_size:
    model.add(Lambda(simple_context,
                     mask = lambda inputs, mask: mask[:,maxlen_x:],
                     output_shape = lambda input_shape: ( input_shape[0],maxlen_y, 2*(rnn_size - activation_rnn_size)),
                     name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                kernel_regularizer=regularizer, bias_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 59, 100)           2972800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 512)           1255424   
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 59, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 59, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 59, 512)           2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 59, 512)           0         
__________

In [40]:
from keras.optimizers import Adam

In [55]:

#optimizer = Adam(lr=0.0002)
#K.set_value(model.optimizer.lr,np.float32(LR))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [84]:
from keras.layers.core import Reshape

In [56]:
model.fit(X_train,Y_train,epochs=3, batch_size=128)

Epoch 1/3
1780/1780 [==============================] - 303s 170ms/step - loss: 7.8262
Epoch 2/3
1780/1780 [==============================] - 307s 173ms/step - loss: 5.6092
Epoch 3/3
1780/1780 [==============================] - 294s 165ms/step - loss: 5.4654


In [57]:
test=model.predict(X_test)

In [202]:
test

array([[[4.3094853e-01, 1.7662477e-03, 1.9458616e-02, ...,
         3.0865652e-09, 3.1480067e-09, 3.2910128e-09],
        [4.3094870e-01, 1.7662467e-03, 1.9458607e-02, ...,
         3.0865666e-09, 3.1479961e-09, 3.2910141e-09],
        [4.3094870e-01, 1.7662467e-03, 1.9458607e-02, ...,
         3.0865666e-09, 3.1479961e-09, 3.2910141e-09],
        ...,
        [4.3094853e-01, 1.7662477e-03, 1.9458616e-02, ...,
         3.0865652e-09, 3.1480067e-09, 3.2910128e-09],
        [4.3094879e-01, 1.7662471e-03, 1.9458620e-02, ...,
         3.0865555e-09, 3.1479908e-09, 3.2910026e-09],
        [4.3094853e-01, 1.7662477e-03, 1.9458616e-02, ...,
         3.0865652e-09, 3.1480067e-09, 3.2910128e-09]],

       [[4.3095508e-01, 1.7662384e-03, 1.9458709e-02, ...,
         3.0863825e-09, 3.1477965e-09, 3.2907930e-09],
        [4.3095508e-01, 1.7662384e-03, 1.9458709e-02, ...,
         3.0863825e-09, 3.1477965e-09, 3.2907930e-09],
        [4.3095508e-01, 1.7662384e-03, 1.9458709e-02, ...,
         3.086

In [190]:
test1.shape

(9, 29727)

In [191]:
test2=test1[7]

In [192]:
test2

array([4.3095881e-01, 1.7662358e-03, 1.9458774e-02, ..., 3.0862857e-09,
       3.1476974e-09, 3.2906771e-09], dtype=float32)

In [193]:
test3=dict(zip(test2, range(len(test2))))

In [194]:
a=sorted(test3.keys(),reverse=True)

In [195]:
test3[a[0]]

0

In [196]:
b=[word for word in word_index if word_index[word]==1]

In [197]:
b

['the']